In [1]:
import cohere
import os
import time
import openai
import configparser

In [4]:
config_path = "./config.ini"
config = configparser.ConfigParser()
config.read(config_path)
cohere_api_key = config.get('cohere', 'api_key')
openai_api_key = config.get('openai', 'api_key')

['./config.ini']

In [2]:
co = cohere.Client("")
openai.api_key = ""

In [4]:
text ="""It's an exciting day for the development community. Cohere's state-of-the-art language AI is now available through Amazon SageMaker. This makes it easier for developers to deploy Cohere's pre-trained generation language model to Amazon SageMaker, an end-to-end machine learning (ML) service. Developers, data scientists, and business analysts use Amazon SageMaker to build, train, and deploy ML models quickly and easily using its fully managed infrastructure, tools, and workflows.
At Cohere, the focus is on language. The company's mission is to enable developers and businesses to add language AI to their technology stack and build game-changing applications with it. Cohere helps developers and businesses automate a wide range of tasks, such as copywriting, named entity recognition, paraphrasing, text summarization, and classification. The company builds and continually improves its general-purpose large language models (LLMs), making them accessible via a simple-to-use platform. Companies can use the models out of the box or tailor them to their particular needs using their own custom data.
Developers using SageMaker will have access to Cohere's Medium generation language model. The Medium generation model excels at tasks that require fast responses, such as question answering, copywriting, or paraphrasing. The Medium model is deployed in containers that enable low-latency inference on a diverse set of hardware accelerators available on AWS, providing different cost and performance advantages for SageMaker customers.
"""

In [3]:
co.summarize(text, model='summarize-medium', length='short', extractiveness='high')

NameError: name 'text' is not defined

In [88]:
model_engine = "gpt-3.5-turbo"
prompt = {"role":"system", "content":"Categorize the following text using maximum five words: " + text}
response = openai.ChatCompletion.create(model = model_engine, messages = [prompt])["choices"][0]["message"]["content"]
response

"Cohere's language AI on SageMaker."

In [5]:
def generate_summarization(texts, model_engine = "gpt-3.5-turbo"):
    main_text = "Use a minimum of one word and a maximum of three words to describe the commonality between the following texts which are separated by the > symbol, shorter answers are preferred:"
    for text in texts:
        main_text += ">" + text
    prompt = {"role":"system", "content": main_text}
    return openai.ChatCompletion.create(model = model_engine, messages = [prompt])["choices"][0]["message"]["content"]

In [6]:
def walkDir(path):
    # recursively walk through dir to get all markdown files
    # returns a dictionary where {filename1: content1, filename2: content2, ...}
    summaries = {}
    # only getting a subset of the directories and files because it's a lot
    for item in os.listdir(path):
        print(item)
        content = os.path.join(path, item)
        if os.path.isdir(content):
            summaries.update(walkDir(content))
        elif content.endswith(".md"):
            fp = open(content)
            not_done = True
            while not_done:
                try:
                    summaries[item] = co.summarize("".join(fp.readlines()), model='summarize-xlarge', length='short', extractiveness='high').summary
                    not_done = False
                except cohere.CohereAPIError:
                    print("API limit reached trying again in 5 seconds")
                    time.sleep(5)

            fp.close()
    return summaries

In [135]:
arch_path = "/Users/kaanankharwa/Desktop/my-second-brain/Architecture"
arch_summarise = walkDir(arch_path)

Architecture.md
Library as space separation.md
Biophilic design.md
Bridges.md


In [136]:
arch_summarise

{'Architecture.md': 'Architecture is a field that entertain me. I like the way art (in the sens of beauty) and engineering is tied together to create such majestic structures and how ancient architecture is.',
 'Library as space separation.md': 'A beautfiul yet simple way to combine space, beauty, and usability of a library is to use it as an ark to separate rooms from each other without having a plain separation. This is very elegant, especialy if filled with old and rare books.',
 'Biophilic design.md': 'Biophilic design is a concept that is used in interior and building design to increase occupant connectivity to the natural environment through the use of direct and indirect nature, space, and place conditions.',
 'Bridges.md': 'One documentary about the Pont Neuf in Paris just learned me some great things about architecture and challenged me and my thoughts about everyday things we use such as running water, sidewalk and bridges construction methods.'}

In [141]:
ar_path = "/Users/kaanankharwa/Desktop/my-second-brain/Augmented reality"
ar_summaries = walkDir(ar_path)

Augmented reality.md
AR devices.md
Mixed reality.md
No-code tools for AR.md


In [138]:
generate_summarization(list(arch_summarise.values()))

'Architecture and design.'

In [142]:
ar_summaries

{'Augmented reality.md': "Augmented reality (AR) is an interactive experience where real-world environment is augmented by virtual and computer-generated perceptual information. It is different than virtual reality (VR) where VR is a 100% computer-generated environment that you're evolving in.",
 'AR devices.md': 'There are four types of AR devices available so far : Heads up displays (HUD) Holographics displays Smart Glasses Handheld',
 'Mixed reality.md': 'Mixed reality is in between the field of Augmented reality and Virtual reality, where the reality is mixed with computer-generated elements.',
 'No-code tools for AR.md': "For now I didn't tried any of these. Just bookmarked it and share it with my students for their projects."}

In [143]:
generate_summarization(list(ar_summaries.values()))

'Immersive technology.'

In [144]:
generate_summarization(list(ar_summaries.values()) + list(arch_summarise.values()))

'Design and architecture.'

In [7]:
def walkDirSummary(path):
    # recursively walk through dir to get all markdown files
    # returns a dictionary where {filename1: content1, filename2: content2, ...}
    summaries = {}
    # only getting a subset of the directories and files because it's a lot
    for item in os.listdir(path):
        print(item)
        content = os.path.join(path, item)
        if os.path.isdir(content):
            summaries.update(walkDir(content))
        elif content.endswith(".md"):
            fp = open(content)
            not_done = True
            while not_done:
                try:
                    text = co.summarize("".join(fp.readlines()), model='summarize-xlarge', length='short', extractiveness='high').summary
                    summaries[item] = generate_summarization([text])
                    not_done = False
                except cohere.CohereAPIError:
                    print("API limit reached trying again in 5 seconds")
                    time.sleep(5)

            fp.close()
    return summaries

In [9]:
ar_path = "/Users/kaanankharwa/Desktop/my-second-brain/Augmented reality"
ar_word = walkDirSummary(ar_path)

Augmented reality.md


RateLimitError: You exceeded your current quota, please check your plan and billing details.